In [2]:
from tensorflow.keras.layers import Lambda, Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import tensorflow as tf
import numpy as np
import cv2
from scipy.spatial import distance
from pathlib import Path
import os

In [3]:

def batchnorm(x, name):
    return BatchNormalization(momentum=0.9, epsilon=2e-5, name=name)(x)

def res_block(x, f, block_name, downscale=False):
    s = 2 if downscale else 1
    shortcut = x
    if downscale:
        shortcut = Conv2D(f, 1, strides=s, use_bias=False, padding="valid", name=block_name+"_conv1sc")(shortcut)
        shortcut = BatchNormalization(momentum=0.9, epsilon=2e-5, name=block_name+"_sc")(shortcut)
    
    x = batchnorm(x, name=block_name+"_bn1")
    x = ZeroPadding2D(1)(x)
    x = Conv2D(f, 3, use_bias=False, padding="valid", name=block_name+"_conv1")(x)
    x = batchnorm(x, name=block_name+"_bn2")
    x = PReLU(shared_axes=[1, 2], name=block_name+"_relu1")(x)
    x = ZeroPadding2D(1)(x)
    x = Conv2D(f, 3, strides=s, use_bias=False, padding="valid", name=block_name+"_conv2")(x)
    x = batchnorm(x, name=block_name+"_bn3")
    
    out = Add()([x, shortcut])
    return out

def LResNet100E_IR(weights_path=None):
    inp = Input((112,112,3))
    minusscalar0 = Lambda(lambda x: x - 127.5, name="minusscalar0")(inp) # "name": "_minusscalar0"
    mulscalar0 = Lambda(lambda x: x / 128, name="mulscalar0")(minusscalar0) # "name": "_mulscalar0"
    
    conv0 = ZeroPadding2D(1)(mulscalar0)
    conv0 = Conv2D(64, 3, padding="valid", use_bias=False, name="conv0")(conv0) #conv0_weight
    bn0 = batchnorm(conv0, name="bn0") # bn0_gamma, bn0_bate, bn0_moving_mean, bn0_moving_var
    relu0 = PReLU(shared_axes=[1, 2], name="relu0")(bn0) # relu0_gamma
    
    stage1 = res_block(relu0, 64, "stage1_unit1", True)
    for i in range(2, 4):
        stage1 = res_block(stage1, 64, f"stage1_unit{str(i)}", False)
      
    stage2 = res_block(stage1, 128, "stage2_unit1", True)
    for i in range(2, 14):
        stage2 = res_block(stage2, 128, f"stage2_unit{str(i)}", False)
      
    stage3 = res_block(stage2, 256, "stage3_unit1", True)
    for i in range(2, 31):
        stage3 = res_block(stage3, 256, f"stage3_unit{str(i)}", False)
      
    stage4 = res_block(stage3, 512, "stage4_unit1", True)
    for i in range(2, 4):
        stage4 = res_block(stage4, 512, f"stage4_unit{str(i)}", False)
      
    bn1 = batchnorm(stage4, name="bn1")
    dropout0 = Dropout(0.4, name="dropout0")(bn1)
    permute = Permute((3,1,2))(dropout0)
    flat = Flatten()(permute)
    pre_fc1 = Dense(512, name="pre_fc1")(flat)
    fc1 = batchnorm(pre_fc1, name="fc1")
    
    model = Model(inp, fc1, name="LResNet100E_IR")
    if weights_path is not None:
        model.load_weights(weights_path)
    return model

In [9]:
def resize_tensor(size):
    input_tensor = Input((None, None, 3)) 
    output_tensor = Lambda(lambda x: tf.image.resize(x, [size, size]))(input_tensor)  
    return Model(input_tensor, output_tensor)

def l2_norm():            
    input_tensor = Input((512,))
    output_tensor = Lambda(lambda x: K.l2_normalize(x))(input_tensor)
    return Model(input_tensor, output_tensor)

In [18]:
input_tensor = Input((None, None, 3))
input_resolution = 112
weights_path = "C:\\Users\\jason\\face_toolbox_keras-master\\models\\verifier\\insightface\\lresnet100e_ir_keras.h5"

resize_layer = resize_tensor(size=input_resolution)
l2_normalize = l2_norm()
lresnet100e_ir = LResNet100E_IR(weights_path=weights_path)
output_tensor = l2_normalize(lresnet100e_ir(resize_layer(input_tensor)))  

In [19]:
model = Model(input_tensor, output_tensor)

In [20]:
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, None, None, 3)]   0         
_________________________________________________________________
model_6 (Functional)         (None, 112, 112, 3)       0         
_________________________________________________________________
LResNet100E_IR (Functional)  (None, 512)               65225792  
_________________________________________________________________
model_7 (Functional)         (None, 512)               0         
Total params: 65,225,792
Trainable params: 65,156,160
Non-trainable params: 69,632
_________________________________________________________________


In [21]:
lresnet100e_ir.summary()

Model: "LResNet100E_IR"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
minusscalar0 (Lambda)           (None, 112, 112, 3)  0           input_15[0][0]                   
__________________________________________________________________________________________________
mulscalar0 (Lambda)             (None, 112, 112, 3)  0           minusscalar0[0][0]               
__________________________________________________________________________________________________
zero_padding2d_198 (ZeroPadding (None, 114, 114, 3)  0           mulscalar0[0][0]                 
_____________________________________________________________________________________

__________________________________________________________________________________________________
stage3_unit3_bn2 (BatchNormaliz (None, 14, 14, 256)  1024        stage3_unit3_conv1[0][0]         
__________________________________________________________________________________________________
stage3_unit3_relu1 (PReLU)      (None, 14, 14, 256)  256         stage3_unit3_bn2[0][0]           
__________________________________________________________________________________________________
zero_padding2d_236 (ZeroPadding (None, 16, 16, 256)  0           stage3_unit3_relu1[0][0]         
__________________________________________________________________________________________________
stage3_unit3_conv2 (Conv2D)     (None, 14, 14, 256)  589824      zero_padding2d_236[0][0]         
__________________________________________________________________________________________________
stage3_unit3_bn3 (BatchNormaliz (None, 14, 14, 256)  1024        stage3_unit3_conv2[0][0]         
__________

zero_padding2d_275 (ZeroPadding (None, 16, 16, 256)  0           stage3_unit23_bn1[0][0]          
__________________________________________________________________________________________________
stage3_unit23_conv1 (Conv2D)    (None, 14, 14, 256)  589824      zero_padding2d_275[0][0]         
__________________________________________________________________________________________________
stage3_unit23_bn2 (BatchNormali (None, 14, 14, 256)  1024        stage3_unit23_conv1[0][0]        
__________________________________________________________________________________________________
stage3_unit23_relu1 (PReLU)     (None, 14, 14, 256)  256         stage3_unit23_bn2[0][0]          
__________________________________________________________________________________________________
zero_padding2d_276 (ZeroPadding (None, 16, 16, 256)  0           stage3_unit23_relu1[0][0]        
__________________________________________________________________________________________________
stage3_uni